In [1]:
# imports
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [2]:
display(train.head())
display(test.head())

,id,bone_length,rotting_flesh,hair_length,has_soul,color,type
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul
3,4,0.776652,0.508723,0.636766,0.884464,black,Ghoul
4,5,0.566117,0.875862,0.418594,0.636438,green,Ghost


,id,bone_length,rotting_flesh,hair_length,has_soul,color
0,3,0.471774,0.387937,0.706087,0.698537,black
1,6,0.427332,0.645024,0.565558,0.451462,white
2,9,0.549602,0.491931,0.660387,0.449809,black
3,10,0.638095,0.682867,0.471409,0.356924,white
4,13,0.361762,0.583997,0.377256,0.276364,black


In [3]:
y = train['type']
train.drop(columns='type',axis=1,inplace=True)
print(train.shape, y.shape)

(371, 6) (371,)


In [4]:
X = pd.get_dummies(train, drop_first=True)
X.head()

,id,bone_length,rotting_flesh,hair_length,has_soul,color_blood,color_blue,color_clear,color_green,color_white
0,0,0.354512,0.350839,0.465761,0.781142,0,0,1,0,0
1,1,0.575560,0.425868,0.531401,0.439899,0,0,0,1,0
2,2,0.467875,0.354330,0.811616,0.791225,0,0,0,0,0
3,4,0.776652,0.508723,0.636766,0.884464,0,0,0,0,0
4,5,0.566117,0.875862,0.418594,0.636438,0,0,0,1,0


In [5]:
test = pd.get_dummies(test, drop_first=True)
test.head()

,id,bone_length,rotting_flesh,hair_length,has_soul,color_blood,color_blue,color_clear,color_green,color_white
0,3,0.471774,0.387937,0.706087,0.698537,0,0,0,0,0
1,6,0.427332,0.645024,0.565558,0.451462,0,0,0,0,1
2,9,0.549602,0.491931,0.660387,0.449809,0,0,0,0,0
3,10,0.638095,0.682867,0.471409,0.356924,0,0,0,0,1
4,13,0.361762,0.583997,0.377256,0.276364,0,0,0,0,0


In [6]:
# drop id
X.drop(columns='id',axis=1,inplace=True)
test_ids = test['id']
test.drop(columns='id',axis=1,inplace=True)

X.shape, test.shape

((371, 9), (529, 9))

In [7]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# initiate model
model_to_set = OneVsRestClassifier(LogisticRegression(penalty = 'l2', max_iter = 1000))

# create parameter options
parameters = {
    "estimator__C": [0.1, 0.5, 0.7, 1.0, 1.2, 2, 5, 10, 20, 100],
    "estimator__solver": ['newton-cg', 'lbfgs', 'sag'],
    "estimator__multi_class" : ['multinomial', 'ovr']
}

# Cross validation
model_tunning = GridSearchCV(model_to_set, param_grid=parameters,
                             cv = 2)

model_tunning.fit(X, y)

print(model_tunning.best_score_)
print(model_tunning.best_params_)

0.7088948787061995
{'estimator__C': 100, 'estimator__multi_class': 'multinomial', 'estimator__solver': 'newton-cg'}


In [8]:
# create model
ovr = OneVsRestClassifier(LogisticRegression(penalty = "l2", C = 100, multi_class = 'multinomial', solver = 'newton-cg'))
# train model
ovr_fitted = ovr.fit(X, y)
# predict test_data
res = ovr_fitted.predict(test)

In [9]:
# save predictions
sample_data = pd.read_csv("sample_submission.csv")
sample_data['id'] = test_ids
sample_data['type'] = res
sample_data.to_csv('prediction1.csv', index = False)